# Preprocessing Markus

In [ ]:
import pandas as pd
path=""
file="Biooekonomie_muss_2022-12-25_18-11-49.csv"

### removing duplicates

In [ ]:
df=pd.read_csv(r""+path+file,header=0, names=columns, na_values='n.v.')
df.drop_duplicates(subset=[ 'Name', 'BVD_ID', 'PLZ', 'Ort',
        'Strasse', 'HNR', 'HNR_Zusatz',
        'web_adresse', 'Anzahl_Mitarbeiter',
        'Anzahl_Mitarbeiter_zuletzt_Jahr',
        'Anzahl_Mitarbeiter_zuletzt',
        'Tochtergesellschaften',
        'Gesellschafter',
        'Unternehmen_im_Konzern',
        'Anzahl_Geschaeftsfuehrer',
        'Umsatz_tsd',
        'Kapital', 'Gruendung',
        'Gesellschafterkapital',
        'Rechtsform', 'Gesellschafter2',
        'Gesellschafter_3',
        'WZ_2008_4_main',
        'Vergleichsgruppe_Name',
        'WZ_2008_4_main_description',
        'WZ_2008_3_main_description',
        'WZ_2008_3_main',
        'WZ_2008_1_main_description',
        'WZ_2008_1_main',
        'WZ_2008_main_description',
        'WZ_2008_main',
        'Vergleichsgruppe_size', 'Allgemeines',
        'Taetigkeitsbeschreibung', 'Crefonummer', 'UID_Nummer',
        'Steuernummer', 'Handelsregisternummer', 'Handelsregisterart',
        'Registernummer', 'Amtsgericht_PLZ',
        'MAN_position',
        'MAN_Crefonummer',
        'MAN_typ',
        'MAN_Anrede', 'MAN_Titel',
        'MAN_Name', 'MAN_Geschlecht',
        'MAN_Funktion', 'MAN_Geburtsjahr',
        'MAN_geschaeftsfuehrender_Gesellschafter',
        'MAN_Funktion_Mailings',
        'MAN_Strasse', 'MAN_HNR',
        'MAN_HNR_Zusatz', 'MAN_Ort',
        'MAN_PLZ', 'MAN_Bundesland',
        'MAN_Land',
        'NACE_main',
        'NACE_main_code',
        'NACE_minor',
        'NACE_minor_code'],keep="first", inplace=True)


### geocoding

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep
geocoder = Nominatim(user_agent = 'selzner@time.rwth-aachen.de')

try:
    
    df=pd.read_csv(r"C:\Users\pauls\OneDrive - rwth-aachen.de\01_BIOÖKONOMIE\AP 1\AP 1.2\Markus_dataset_geolocated_temp.csv",index_col=0)
    x=len(df[df.lat.notna()])
    print(f"file loaded. {x} companies already geocoded. continuing geocoding")
except:
    print("file not found. starting from beginning")
    df= pd.read_csv(r""+path+file, index_col=0)
    df["lat"]=np.nan
    df["lon"]=np.nan
    x=0

for i,row in df.iterrows():
    if (row.Name==" ") or (df.lat.notna()[i]):
        continue
    current_adress={"street":f"{row.HNR} {row.Strasse}", "city":row.Ort, "country" : "Germany", "postalcode" : row.PLZ}
    try:
        geolocation=geocoder.geocode(current_adress, country_codes="de")
    except:
        df.loc[i,"lat"]="timeout"
        df.loc[i,"lon"]="timeout"
        sleep(5)
        print(f"timeout error at {x}")
        
    try:
        df.loc[i,"lat"]=geolocation.latitude
        df.loc[i,"lon"]=geolocation.longitude
    except:
        df.loc[i,"lat"]="error"
        df.loc[i,"lon"]="error"
    sleep(1)
    x+=1
    if i%1000==0:
        print(f"{i} of {len(df)} % abgeschlossen.")
        df.to_csv(r"C:\Users\pauls\OneDrive - rwth-aachen.de\01_BIOÖKONOMIE\AP 1\AP 1.2\Markus_dataset_geolocated_temp.csv")
print("Geocodierung abgeschlossen.")
print(f'{df[df.lat=="error"]} Unternehmen konnten nicht geolokalisiert werden.')
df.to_csv(r"C:\Users\pauls\OneDrive - rwth-aachen.de\01_BIOÖKONOMIE\AP 1\AP 1.2\Markus_dataset_geolocated.csv")

# Preprocessing for SQL

In [7]:
import pandas as pd

In [81]:
df=pd.read_csv(r"C:\Users\PaulSelzner\OneDrive - rwth-aachen.de\01_BIOÖKONOMIE\AP 1\AP 1.2\Markus_dataset_geolocated.csv",index_col=0, na_values=" ",dtype={'id': int,'Name': str,'BVD_ID': str,'PLZ': str,'Ort': str,'Strasse': str,'HNR': str,'HNR_Zusatz': str,'web_adresse': str,'Anzahl_Mitarbeiter': str,'Anzahl_Mitarbeiter_zuletzt_Jahr': str,'Anzahl_Mitarbeiter_zuletzt': str,'Tochtergesellschaften': str,'Gesellschafter': str,'Unternehmen_im_Konzern': str,'Anzahl_Geschaeftsfuehrer': str,'Umsatz_tsd': str,'Kapital': str,'Gruendung': str,'Gesellschafterkapital': str,'Rechtsform': str,'Gesellschafter2': str,'Gesellschafter_3': str,'WZ_2008_4_main': str,'Vergleichsgruppe_Name': str,'WZ_2008_4_main_description': str,'WZ_2008_3_main_description': str,'WZ_2008_3_main': str,'WZ_2008_1_main_description': str,'WZ_2008_1_main': str,'WZ_2008_main_description': str,'WZ_2008_main': str,'Vergleichsgruppe_size': str,'Allgemeines': str,'Taetigkeitsbeschreibung': str,'Crefonummer': str,'UID_Nummer': str,'Steuernummer': str,'Handelsregisternummer': str,'Handelsregisterart': str,'Registernummer': str,'Amtsgericht_PLZ': str,'MAN_position': str,'MAN_Crefonummer': str,'MAN_typ': str,'MAN_Anrede': str,'MAN_Titel': str,'MAN_Name': str,'MAN_Geschlecht': str,'MAN_Funktion': str,'MAN_Geburtsjahr': str,'MAN_geschaeftsfuehrender_Gesellschafter': str,'MAN_Funktion_Mailings': str,'MAN_Strasse': str,'MAN_HNR': str,'MAN_HNR_Zusatz': str,'MAN_Ort': str,'MAN_PLZ': str,'MAN_Bundesland': str,'MAN_Land': str,'NACE_main': str,'NACE_main_code': str,'NACE_minor': str,'NACE_minor_code': str,'lat': str,'lon': str})

In [82]:
# Function to remove dots
def remove_dots(text):
    if type(text)!=str:
        text=str(text)
    return text.replace('.', '')

cols_with_dots=['Anzahl_Mitarbeiter', 'Anzahl_Mitarbeiter_zuletzt_Jahr',
       'Anzahl_Mitarbeiter_zuletzt', 'Tochtergesellschaften', 'Gesellschafter',
       'Unternehmen_im_Konzern', 'Anzahl_Geschaeftsfuehrer', 'Umsatz_tsd',
       'Kapital', 'Gesellschafterkapital','Gesellschafter2', 'Gesellschafter_3', 'Crefonummer', 'UID_Nummer',
       'Steuernummer', 'Handelsregisternummer', 'Amtsgericht_PLZ','NACE_main', 'NACE_main_code', 'Vergleichsgruppe_size']
for col in cols_with_dots:
    #print(col)
    df[col]=df[col].apply(remove_dots)
                   

# Function to transform date format
def transform_date(date):
    if type(date)!=str:
        date=str(date)
    parts = date.split('/')
    if len(parts)==3:
        return f'{parts[2]}-{parts[1]}-{parts[0]}'
    if len(date)==4:
        return f'{date}-01-01'
    return 'NaN'

# Apply function to the 'Date' column
df['Gruendung'] = df['Gruendung'].apply(transform_date)        


def clean_coordinates(coord):
    if coord=="timeout" or coord=="error":
        return "NaN"
    return coord
df["lat"]=df["lat"].apply(clean_coordinates)
df["lon"]=df["lon"].apply(clean_coordinates)


def assing_nan_values(cell):
    if str(cell)=="" or str(cell)=="nan":
        return "NaN"
    return cell

for col in ['id', 'Name', 'BVD_ID', 'PLZ', 'Ort', 'Strasse', 'HNR', 'HNR_Zusatz',
       'web_adresse', 'Anzahl_Mitarbeiter', 'Anzahl_Mitarbeiter_zuletzt_Jahr',
       'Anzahl_Mitarbeiter_zuletzt', 'Tochtergesellschaften', 'Gesellschafter',
       'Unternehmen_im_Konzern', 'Anzahl_Geschaeftsfuehrer', 'Umsatz_tsd',
       'Kapital', 'Gruendung', 'Gesellschafterkapital', 'Rechtsform',
       'Gesellschafter2', 'Gesellschafter_3', 'WZ_2008_4_main',
       'Vergleichsgruppe_Name', 'WZ_2008_4_main_description',
       'WZ_2008_3_main_description', 'WZ_2008_3_main',
       'WZ_2008_1_main_description', 'WZ_2008_1_main',
       'WZ_2008_main_description', 'WZ_2008_main', 'Vergleichsgruppe_size',
       'Allgemeines', 'Taetigkeitsbeschreibung', 'Crefonummer', 'UID_Nummer',
       'Steuernummer', 'Handelsregisternummer', 'Handelsregisterart',
       'Registernummer', 'Amtsgericht_PLZ','NACE_main', 'NACE_main_code',
       'NACE_minor', 'NACE_minor_code', 'lat', 'lon']:
    df[col]=df[col].apply(assing_nan_values)


### adding further information


In [ ]:
#Data which is needed for some apache superset visualizations
#ISO 3166-2 codes (Bundesland)
##ISO 3166-1 alpha 2 code (DE)
# polygon columns (polyline or JSON)
# network table start lat/lon end lat/lon


In [152]:
markus_companies=df[['id', 'Name', 'BVD_ID', 'PLZ', 'Ort', 'Strasse', 'HNR', 'HNR_Zusatz',
       'web_adresse', 'Anzahl_Mitarbeiter', 'Anzahl_Mitarbeiter_zuletzt_Jahr',
       'Anzahl_Mitarbeiter_zuletzt', 'Tochtergesellschaften', 'Gesellschafter',
       'Unternehmen_im_Konzern', 'Anzahl_Geschaeftsfuehrer', 'Umsatz_tsd',
       'Kapital', 'Gruendung', 'Gesellschafterkapital', 'Rechtsform',
       'Gesellschafter2', 'Gesellschafter_3', 'WZ_2008_4_main',
       'Vergleichsgruppe_Name', 'WZ_2008_4_main_description',
       'WZ_2008_3_main_description', 'WZ_2008_3_main',
       'WZ_2008_1_main_description', 'WZ_2008_1_main',
       'WZ_2008_main_description', 'WZ_2008_main', 'Vergleichsgruppe_size',
       'Allgemeines', 'Taetigkeitsbeschreibung', 'Crefonummer', 'UID_Nummer',
       'Steuernummer', 'Handelsregisternummer', 'Handelsregisterart',
       'Registernummer', 'Amtsgericht_PLZ','NACE_main', 'NACE_main_code',
       'NACE_minor', 'NACE_minor_code', 'lat', 'lon']][df.Name!="NaN"].copy()

markus_person=df[['id','BVD_ID', 'Handelsregisternummer','MAN_position', 'MAN_Crefonummer',
       'MAN_typ', 'MAN_Anrede', 'MAN_Titel', 'MAN_Name', 'MAN_Geschlecht',
       'MAN_Funktion', 'MAN_Geburtsjahr',
       'MAN_geschaeftsfuehrender_Gesellschafter', 'MAN_Funktion_Mailings',
       'MAN_Strasse', 'MAN_HNR', 'MAN_HNR_Zusatz', 'MAN_Ort', 'MAN_PLZ',
       'MAN_Bundesland', 'MAN_Land']].copy()

ids=markus_person.id.unique()
for current_id in ids:
    current_bvd=markus_person.loc[markus_person.id==current_id,"BVD_ID"].iloc[0]
    markus_person.loc[markus_person.id==current_id,"BVD_ID"]=current_bvd
    
for current_id in ids:
    current_HRN=markus_person.loc[markus_person.id==current_id,"Handelsregisternummer"].iloc[0]
    markus_person.loc[markus_person.id==current_id,"Handelsregisternummer"]=current_HRN
    

markus_companies.to_csv(r"C:\Users\PaulSelzner\OneDrive - rwth-aachen.de\05_Rolle\03_Entwicklung\Metabase\Daten\Markus_companies.csv", index=False)
markus_person.to_csv(r"C:\Users\PaulSelzner\OneDrive - rwth-aachen.de\05_Rolle\03_Entwicklung\Metabase\Daten\markus_person.csv", index=False)


In [151]:
markus_companies[(markus_companies.Name != "NaN") ]

,id,Name,BVD_ID,PLZ,Ort,Strasse,HNR,HNR_Zusatz,web_adresse,Anzahl_Mitarbeiter,...,Handelsregisternummer,Handelsregisterart,Registernummer,Amtsgericht_PLZ,NACE_main,NACE_main_code,NACE_minor,NACE_minor_code,lat,lon
0,0,Santander Consumer Bank Aktiengesellschaft,DEFEB13180,41061,Mönchengladbach,Santander-Platz,1,NaN,www.santander.de,2963,...,41061 HRB 1747,HRB,1747,41061,6419,Kreditinstitute (ohne Spezialkreditinstitute),NaN,NaN,51.1952218,6.42310693442816
18,1,"Yamaha Motor Europe N.V., Niederlassung Deutsc...",DE5270256067,41468,Neuss,Hansemannstr.,12,NaN,www.yamaha-motor.eu,46,...,41460 HRB 18872,HRB,18872,41460,4669,Großhandel mit sonstigen Maschinen und Ausrüst...,4646,"Großhandel mit pharmazeutischen, medizinischen...",51.1603288,6.772097715571365
20,2,Lidl Vertriebs-GmbH & Co. KG,DE5270137212,41515,Grevenbroich,Lise-Meitner-Str.,4,NaN,www.lidl.de,2,...,41061 HRA 4290,HRA,4290,41061,4711,"Einzelhandel mit Waren verschiedener Art, Haup...",4638,Großhandel mit sonstigen Nahrungs- und Genussm...,51.08334535,6.625341387575065
22,3,Trianel GmbH,DE5010267951,52070,Aachen,Krefelder Str.,203,NaN,www.trianel.com,325,...,52070 HRB 7729,HRB,7729,52070,3511,Elektrizitätserzeugung,8299,Erbringung sonstiger wirtschaftlicher Dienstle...,50.7919344,6.097041461128711
28,4,Basell Polyolefine GmbH,DE5190001617,50389,Wesseling,Brühler Str.,60,NaN,www.basell-wesseling.de,2343,...,50939 HRB 45129,HRB,45129,50939,2051,Herstellung von pyrotechnischen Erzeugnissen,2016,Herstellung von Kunststoffen in Primärformen,50.8332731,6.965967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134077,68071,ÜberLink GmbH,DE5050749587,40667,Meerbusch,Ingrid-von-Schmettow-Str.,8,NaN,NaN,2,...,40227 HRB 74362,HRB,74362,40227,9609,Erbringung von sonstigen Dienstleistungen a n g,6202,Erbringung von Beratungsleistungen auf dem Geb...,51.25081155,6.6857396401958855
134079,68072,Übermittagsbetreuung an der RBS e.V.,DE5270227573,41515,Grevenbroich,Bergheimer Str.,49,-51,NaN,3,...,41236 VR 4853,VR,4853,41236,NaN,NaN,NaN,NaN,51.08723485,6.579848534578377
134082,68073,Übersetzungsmanufaktur e.K. Inhaberin: Michael...,DE5230402475,41236,Mönchengladbach,Wickrather Str.,31,-33,www.uebersetzungs-manufaktur.de,NaN,...,41061 HRA 7421,HRA,7421,41061,7430,Übersetzen und Dolmetschen,NaN,NaN,51.1613316,6.4403238
134083,68074,Ückerather Schwadlappe e.V.,DE5270231705,41542,Dormagen,Kemmerlingstr.,21,NaN,NaN,3,...,41460 VR 2704,VR,2704,41460,9499,Sonstige Interessenvertretungen und Vereinigun...,9329,Erbringung von Dienstleistungen der Unterhaltu...,51.11108715,6.758778452353814


# Loading INKAR

In [3]:
import pandas as pd
inkar=pd.read_csv(r"Daten\INKAR\inkar_2022\inkar_2022.csv", sep=";")

C:\Users\pauls\AppData\Local\Temp\ipykernel_3000\978636707.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  inkar=pd.read_csv(r"Daten\INKAR\inkar_2022\inkar_2022.csv", sep=";")


In [5]:
inkar.Raumbezug.unique()

array(['Arbeitsmarktregionen', 'BBSR-Mittelbereiche', 'Braunkohlereviere',
       'Bund', 'Bundesländer', 'Europäische Metropolregion',
       'Gemeindeverbände', 'Großstadtregionaler Einzugsbereich',
       'Großstadtregionen', 'IHK-Bezirke', 'Kreise', 'Kreisregionen',
       'Raumordnungsregionen', 'Raumtyp Bezug Lage (Kreise)',
       'Raumtyp Bezug: Lage (Gemeindeverbände)', 'Regierungsbezirke',
       'Regionalstatistischer Raumtyp 17 (RegioStaR17)',
       'Regionalstatistischer Raumtyp 2 (RegioStaR2)',
       'Regionalstatistischer Raumtyp 7 (RegioStaR7)',
       'Regionalstatistischer Raumtyp Gemeindetyp 5 (Regio',
       'Siedlungsstruktureller Kreistyp',
       'Siedlungsstruktureller Kreistyp (Bundesländer)',
       'Siedlungsstruktureller Kreistyp (West-Ost)',
       'Siedlungsstruktureller Regionstyp',
       'Siedlungsstruktureller Regionstyp (Bundesländer)',
       'Siedlungsstruktureller Regionstyp (West-Ost)',
       'Stadt- und Gemeindetyp', 'Stadt- und Gemeindetyp (B

In [7]:
inkar[inkar.Bereich=="Absolutzahlen"].Indikator.unique()

array(['Arbeitslose', 'Bruttoinlandsprodukt in 1000 Euro',
       'Sozialversicherungspflichtig Beschäftigte am Wohnort',
       'Sozialversicherungspflichtig Beschäftigte am Arbeitsort',
       'Bevölkerung (mit Korrektur VZ 1987/Zensus 2011)',
       'Bevölkerung gesamt', 'Bevölkerung weiblich',
       'Bevölkerung männlich',
       'Erwerbsfähige Bevölkerung (15 bis unter 65 Jahre)',
       'Bodenfläche gesamt qkm', 'Erwerbstätige'], dtype=object)

In [10]:
inkar

,Bereich,ID,Indikator,Raumbezug,Kennziffer,Name,Zeitbezug,Wert
0,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,1,Flensburg,1995,"10494,00"
1,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,1,Flensburg,1996,"11921,00"
2,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,1,Flensburg,1997,"13213,00"
3,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,1,Flensburg,1998,"13301,00"
4,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,1,Flensburg,1999,"12048,00"
...,...,...,...,...,...,...,...,...
25323753,Zentrale Orte Monitoring,156270,Kinositzplätze,zentralörtliche Funktion - Oberzentrum,15003000,"Magdeburg, Landeshauptstadt",2017,"5915,00"
25323754,Zentrale Orte Monitoring,156270,Kinositzplätze,zentralörtliche Funktion - Oberzentrum,16051000,"Erfurt, Stadt",2017,"2260,00"
25323755,Zentrale Orte Monitoring,156270,Kinositzplätze,zentralörtliche Funktion - Oberzentrum,16052000,"Gera, Stadt",2017,"1934,00"
25323756,Zentrale Orte Monitoring,156270,Kinositzplätze,zentralörtliche Funktion - Oberzentrum,16053000,"Jena, Stadt",2017,"1560,00"


In [17]:
inkar[inkar.Name=="Aachen"]

,Bereich,ID,Indikator,Raumbezug,Kennziffer,Name,Zeitbezug,Wert
1144,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,45,Aachen,1995,"44907,00"
1145,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,45,Aachen,1996,"49454,00"
1146,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,45,Aachen,1997,"52004,00"
1147,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,45,Aachen,1998,"50678,00"
1148,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,45,Aachen,1999,"48696,00"
...,...,...,...,...,...,...,...,...
20259995,Wirtschaft,17993,Unternehmensinsolvenzen Gläubigerforderungen,Raumordnungsregionen,501,Aachen,2016,"521869,22"
20259996,Wirtschaft,17993,Unternehmensinsolvenzen Gläubigerforderungen,Raumordnungsregionen,501,Aachen,2017,"4684225,56"
20259997,Wirtschaft,17993,Unternehmensinsolvenzen Gläubigerforderungen,Raumordnungsregionen,501,Aachen,2018,"253917,84"
20259998,Wirtschaft,17993,Unternehmensinsolvenzen Gläubigerforderungen,Raumordnungsregionen,501,Aachen,2019,"365986,56"


In [18]:
inkar[inkar.ID==156270]

,Bereich,ID,Indikator,Raumbezug,Kennziffer,Name,Zeitbezug,Wert
25301808,Zentrale Orte Monitoring,156270,Kinositzplätze,Gemeinden,1001000,"Flensburg, Stadt",2017,"2101,00"
25301809,Zentrale Orte Monitoring,156270,Kinositzplätze,Gemeinden,1002000,"Kiel, Landeshauptstadt",2017,"4309,00"
25301810,Zentrale Orte Monitoring,156270,Kinositzplätze,Gemeinden,1003000,"Lübeck, Hansestadt",2017,"1884,00"
25301811,Zentrale Orte Monitoring,156270,Kinositzplätze,Gemeinden,1004000,"Neumünster, Stadt",2017,"2548,00"
25301812,Zentrale Orte Monitoring,156270,Kinositzplätze,Gemeinden,1051001,Albersdorf,2017,"0,00"
...,...,...,...,...,...,...,...,...
25323753,Zentrale Orte Monitoring,156270,Kinositzplätze,zentralörtliche Funktion - Oberzentrum,15003000,"Magdeburg, Landeshauptstadt",2017,"5915,00"
25323754,Zentrale Orte Monitoring,156270,Kinositzplätze,zentralörtliche Funktion - Oberzentrum,16051000,"Erfurt, Stadt",2017,"2260,00"
25323755,Zentrale Orte Monitoring,156270,Kinositzplätze,zentralörtliche Funktion - Oberzentrum,16052000,"Gera, Stadt",2017,"1934,00"
25323756,Zentrale Orte Monitoring,156270,Kinositzplätze,zentralörtliche Funktion - Oberzentrum,16053000,"Jena, Stadt",2017,"1560,00"


In [128]:
for x in dt:
    print(x)

int64
object
object
float64
object
object
float64
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
float64
object
object
object
object
object
object
float64
object
object
object
float64
object
object
object
object
object
object
object
float64
object
object
object


In [107]:
markus_companies.iloc[61:65,10:17]

,Anzahl_Mitarbeiter_zuletzt_Jahr,Anzahl_Mitarbeiter_zuletzt,Tochtergesellschaften,Gesellschafter,Unternehmen_im_Konzern,Anzahl_Geschaeftsfuehrer,Umsatz_tsd
61,nan,nan,nan,nan,nan,nan,nan
62,nan,nan,nan,nan,nan,nan,nan
63,nan,nan,nan,nan,nan,nan,nan
64,nan,nan,nan,nan,nan,nan,nan


In [104]:
markus_companies.loc[585,"Anzahl_Mitarbeiter_zuletzt_Jahr"]

'nan'

In [39]:
markus_test=df[['id', 'Name']][df.Name!=" "].copy()
markus_test.to_csv(r"C:\Users\PaulSelzner\OneDrive - rwth-aachen.de\05_Rolle\03_Entwicklung\Metabase\Daten\markus_test.csv", index=False)
markus_companies_test=df[['id', 'Name', 'BVD_ID', 'PLZ', 'Ort', 'Strasse', 'HNR', 'HNR_Zusatz',
       'web_adresse', 'Anzahl_Mitarbeiter', 'Anzahl_Mitarbeiter_zuletzt_Jahr',
       'Anzahl_Mitarbeiter_zuletzt', 'Tochtergesellschaften', 'Gesellschafter',
       'Unternehmen_im_Konzern', 'Anzahl_Geschaeftsfuehrer', 'Umsatz_tsd',
       'Kapital', 'Gruendung', 'Gesellschafterkapital', 'Rechtsform',
       'Gesellschafter2', 'Gesellschafter_3', 'WZ_2008_4_main',
       'Vergleichsgruppe_Name', 'WZ_2008_4_main_description',
       'WZ_2008_3_main_description', 'WZ_2008_3_main',
       'WZ_2008_1_main_description', 'WZ_2008_1_main',
       'WZ_2008_main_description', 'WZ_2008_main', 'Vergleichsgruppe_size',
       'Allgemeines', 'Taetigkeitsbeschreibung', 'Crefonummer', 'UID_Nummer',
       'Steuernummer', 'Handelsregisternummer', 'Amtsgericht_PLZ','NACE_main', 'NACE_main_code',
       'NACE_minor', 'NACE_minor_code', 'lat', 'lon']][df.Name!=" "].copy()

markus_companies_test.to_csv(r"C:\Users\PaulSelzner\OneDrive - rwth-aachen.de\05_Rolle\03_Entwicklung\Metabase\Daten\markus_companies_test.csv", index=False)


In [29]:
df.Gruendung

0         20/08/1957
1                   
2                   
3                   
4                   
             ...    
134082    04/02/2016
134083    24/05/2012
134084              
134085              
134086    17/12/2015
Name: Gruendung, Length: 97136, dtype: object

In [11]:
markus_companies.columns


Index(['id', 'Name', 'BVD_ID', 'PLZ', 'Ort', 'Strasse', 'HNR', 'HNR_Zusatz',
       'web_adresse', 'Anzahl_Mitarbeiter', 'Anzahl_Mitarbeiter_zuletzt_Jahr',
       'Anzahl_Mitarbeiter_zuletzt', 'Tochtergesellschaften', 'Gesellschafter',
       'Unternehmen_im_Konzern', 'Anzahl_Geschaeftsfuehrer', 'Umsatz_tsd',
       'Kapital', 'Gruendung', 'Gesellschafterkapital', 'Rechtsform',
       'Gesellschafter2', 'Gesellschafter_3', 'WZ_2008_4_main',
       'Vergleichsgruppe_Name', 'WZ_2008_4_main_description',
       'WZ_2008_3_main_description', 'WZ_2008_3_main',
       'WZ_2008_1_main_description', 'WZ_2008_1_main',
       'WZ_2008_main_description', 'WZ_2008_main', 'Vergleichsgruppe_size',
       'Allgemeines', 'Taetigkeitsbeschreibung', 'Crefonummer', 'UID_Nummer',
       'Steuernummer', 'Handelsregisternummer', 'Handelsregisterart',
       'Registernummer', 'Amtsgericht_PLZ', 'NACE_main', 'NACE_main_code',
       'NACE_minor', 'NACE_minor_code', 'lat', 'lon'],
      dtype='object')

In [8]:
df[['id', 'Name']]

,id,Name
0,0,Santander Consumer Bank Aktiengesellschaft
1,0,
2,0,
3,0,
4,0,
...,...,...
134082,68073,Übersetzungsmanufaktur e.K. Inhaberin: Michael...
134083,68074,Ückerather Schwadlappe e.V.
134084,68074,
134085,68074,


In [18]:
markus_companies.loc[0,"Anzahl_Mitarbeiter"]

'2963'

In [1]:
import pandas as pd
markus_companies=pd.read_csv(r"C:\Users\PaulSelzner\OneDrive - rwth-aachen.de\05_Rolle\03_Entwicklung\Metabase\Daten\Markus_companies.csv")

In [2]:
markus_companies

,id,Name,BVD_ID,PLZ,Ort,Strasse,HNR,HNR_Zusatz,web_adresse,Anzahl_Mitarbeiter,...,Handelsregisternummer,Handelsregisterart,Registernummer,Amtsgericht_PLZ,NACE_main,NACE_main_code,NACE_minor,NACE_minor_code,lat,lon
0,0,Santander Consumer Bank Aktiengesellschaft,DEFEB13180,41061,Mönchengladbach,Santander-Platz,1.0,NaN,www.santander.de,2963.0,...,41061 HRB 1747,HRB,1747,41061.0,6419.0,Kreditinstitute (ohne Spezialkreditinstitute),NaN,NaN,51.195222,6.423107
1,1,"Yamaha Motor Europe N.V., Niederlassung Deutsc...",DE5270256067,41468,Neuss,Hansemannstr.,12.0,NaN,www.yamaha-motor.eu,46.0,...,41460 HRB 18872,HRB,18872,41460.0,4669.0,Großhandel mit sonstigen Maschinen und Ausrüst...,4646.0,"Großhandel mit pharmazeutischen, medizinischen...",51.160329,6.772098
2,2,Lidl Vertriebs-GmbH & Co. KG,DE5270137212,41515,Grevenbroich,Lise-Meitner-Str.,4.0,NaN,www.lidl.de,2.0,...,41061 HRA 4290,HRA,4290,41061.0,4711.0,"Einzelhandel mit Waren verschiedener Art, Haup...",4638.0,Großhandel mit sonstigen Nahrungs- und Genussm...,51.083345,6.625341
3,3,Trianel GmbH,DE5010267951,52070,Aachen,Krefelder Str.,203.0,NaN,www.trianel.com,325.0,...,52070 HRB 7729,HRB,7729,52070.0,3511.0,Elektrizitätserzeugung,8299.0,Erbringung sonstiger wirtschaftlicher Dienstle...,50.791934,6.097041
4,4,Basell Polyolefine GmbH,DE5190001617,50389,Wesseling,Brühler Str.,60.0,NaN,www.basell-wesseling.de,2343.0,...,50939 HRB 45129,HRB,45129,50939.0,2051.0,Herstellung von pyrotechnischen Erzeugnissen,2016.0,Herstellung von Kunststoffen in Primärformen,50.833273,6.965967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66972,68071,ÜberLink GmbH,DE5050749587,40667,Meerbusch,Ingrid-von-Schmettow-Str.,8.0,NaN,NaN,2.0,...,40227 HRB 74362,HRB,74362,40227.0,9609.0,Erbringung von sonstigen Dienstleistungen a n g,6202.0,Erbringung von Beratungsleistungen auf dem Geb...,51.250812,6.685740
66973,68072,Übermittagsbetreuung an der RBS e.V.,DE5270227573,41515,Grevenbroich,Bergheimer Str.,49.0,-51,NaN,3.0,...,41236 VR 4853,VR,4853,41236.0,NaN,NaN,NaN,NaN,51.087235,6.579849
66974,68073,Übersetzungsmanufaktur e.K. Inhaberin: Michael...,DE5230402475,41236,Mönchengladbach,Wickrather Str.,31.0,-33,www.uebersetzungs-manufaktur.de,NaN,...,41061 HRA 7421,HRA,7421,41061.0,7430.0,Übersetzen und Dolmetschen,NaN,NaN,51.161332,6.440324
66975,68074,Ückerather Schwadlappe e.V.,DE5270231705,41542,Dormagen,Kemmerlingstr.,21.0,NaN,NaN,3.0,...,41460 VR 2704,VR,2704,41460.0,9499.0,Sonstige Interessenvertretungen und Vereinigun...,9329.0,Erbringung von Dienstleistungen der Unterhaltu...,51.111087,6.758778


In [87]:
import pandas as pd
markus_companies=pd.read_csv(r"C:\Users\pauls\OneDrive - rwth-aachen.de\05_Rolle\03_Entwicklung\Metabase\Daten\Markus_companies_enhanced.csv",
                             index_col=0, na_values=[" ","NaN"],dtype={'id': int,'Name': str,'BVD_ID': str,'PLZ': str,'Ort': str,'Strasse': str,'HNR': str,
                                                               'HNR_Zusatz': str,'web_adresse': str,'Anzahl_Mitarbeiter': int,
                                                               'Anzahl_Mitarbeiter_zuletzt_Jahr': str,'Anzahl_Mitarbeiter_zuletzt': int,
                                                               'Tochtergesellschaften': str,'Gesellschafter': str,'Unternehmen_im_Konzern': str,
                                                               'Anzahl_Geschaeftsfuehrer': str,'Umsatz_tsd': str,'Kapital': str,'Gruendung': str,
                                                               'Gesellschafterkapital': str,'Rechtsform': str,'Gesellschafter2': str,
                                                               'Gesellschafter_3': str,'WZ_2008_4_main': str,'Vergleichsgruppe_Name': str,
                                                               'WZ_2008_4_main_description': str,'WZ_2008_3_main_description': str,'WZ_2008_3_main': str,
                                                               'WZ_2008_1_main_description': str,'WZ_2008_1_main': str,'WZ_2008_main_description': str,
                                                               'WZ_2008_main': str,'Vergleichsgruppe_size': str,'Allgemeines': str,
                                                               'Taetigkeitsbeschreibung': str,'Crefonummer': str,'UID_Nummer': str,
                                                               'Steuernummer': str,'Handelsregisternummer': str,'Handelsregisterart': str,
                                                               'Registernummer': str,'Amtsgericht_PLZ': str,'MAN_position': str,'MAN_Crefonummer': str,
                                                               'MAN_typ': str,'MAN_Anrede': str,'MAN_Titel': str,'MAN_Name': str,'MAN_Geschlecht': str,
                                                               'MAN_Funktion': str,'MAN_Geburtsjahr': str,'MAN_geschaeftsfuehrender_Gesellschafter': str,
                                                               'MAN_Funktion_Mailings': str,'MAN_Strasse': str,'MAN_HNR': str,'MAN_HNR_Zusatz': str,
                                                               'MAN_Ort': str,'MAN_PLZ': str,'MAN_Bundesland': str,'MAN_Land': str,'NACE_main': str,
                                                               'NACE_main_code': str,'NACE_minor': str,'NACE_minor_code': str,'lat': str,'lon': str, 
                                                               'Gemeinde_AGS':str,'Gemeinde':str}, parse_dates=["Gruendung"])

ValueError: Integer column has NA values in column 9

In [31]:
markus_companies.Gruendung

id
1332     1992-05-26
1424     1891-01-01
3169     1982-11-15
3329     2019-12-19
4060     1987-07-17
            ...    
10952    2006-12-19
34254    2001-12-05
12066    2006-11-13
32331    2009-02-18
10577    2011-09-02
Name: Gruendung, Length: 66977, dtype: object

In [38]:
markus_companies[['WZ_2008_4_main','Vergleichsgruppe_Name','WZ_2008_4_main_description',       'WZ_2008_3_main_description', 'WZ_2008_3_main',
       'WZ_2008_1_main_description', 'WZ_2008_1_main',       'WZ_2008_main_description', 'WZ_2008_main']]

,WZ_2008_4_main,Vergleichsgruppe_Name,WZ_2008_4_main_description,WZ_2008_3_main_description,WZ_2008_3_main,WZ_2008_1_main_description,WZ_2008_1_main,WZ_2008_main_description,WZ_2008_main
id,,,,,,,,,
1332,G451,G451 - VL,Handel mit Kraftwagen,Handel mit Kraftfahrzeugen; Instandhaltung und...,G45,HANDEL; INSTANDHALTUNG UND REPARATUR VON KRAFT...,G,Handel mit Kraftwagen mit einem Gesamtgewicht ...,G45110
1424,I562,I562 - VL,Caterer und Erbringung sonstiger Verpflegungsd...,Gastronomie,I56,GASTGEWERBE,I,Event-Caterer,I56210
3169,G451,G451 - VL,Handel mit Kraftwagen,Handel mit Kraftfahrzeugen; Instandhaltung und...,G45,HANDEL; INSTANDHALTUNG UND REPARATUR VON KRAFT...,G,Handel mit Kraftwagen,G45100
3329,G461,G461 - LA,Handelsvermittlung,Großhandel (ohne Handel mit Kraftfahrzeugen),G46,HANDEL; INSTANDHALTUNG UND REPARATUR VON KRAFT...,G,Handelsvermittlung von keramischen Erzeugnisse...,G46152
4060,G477,G477 - VL,Einzelhandel mit sonstigen Gütern (in Verkaufs...,Einzelhandel (ohne Handel mit Kraftfahrzeugen),G47,HANDEL; INSTANDHALTUNG UND REPARATUR VON KRAFT...,G,"Einzelhandel mit Blumen, Pflanzen, Sämereien u...",G47761
...,...,...,...,...,...,...,...,...,...
10952,F432,F432 - ME,Bauinstallation,"Vorbereitende Baustellenarbeiten, Bauinstallat...",F43,BAUGEWERBE,F,"Gas-, Wasser-, Heizungs- sowie Lüftungs- und K...",F43220
34254,M701,M701 - VL,Verwaltung und Führung von Unternehmen und Bet...,Verwaltung und Führung von Unternehmen und Bet...,M70,"ERBRINGUNG VON FREIBERUFLICHEN, WISSENSCHAFTLI...",M,Sonstige Verwaltung und Führung von Unternehme...,M70109
12066,C181,C181 - SM,Herstellung von Druckerzeugnissen,Herstellung von Druckerzeugnissen; Vervielfält...,C18,VERARBEITENDES GEWERBE,C,Drucken a. n. g.,C18120


In [ ]:
df.groupby()

In [69]:
lon1=[]
lat1=[]
lon2=[]
lat2=[]
company1=[]
company2=[]
mitarbeiter1=[]
mitarbeiter2=[]

for x in range(len(markus_companies)):
    print(x)
    bvID=markus_companies.iloc[x].BVD_ID
    wz=markus_companies.iloc[x].WZ_2008_main
    
    temp_table=markus_companies[(markus_companies.WZ_2008_main==wz) & (markus_companies.BVD_ID!=bvID) & markus_companies.lon.notna()].copy()
    if len(temp_table)==0:
        continue
    for y in range(len(temp_table)):
        lon1.append(markus_companies.iloc[x].lon)
        lat1.append(markus_companies.iloc[x].lat)
        lon1.append(temp_table.iloc[y].lon)
        lat1.append(temp_table.iloc[y].lat)
        mitarbeiter1.append(markus_companies.iloc[x].Anzahl_Mitarbeiter)
        mitarbeiter2.append(temp_table.iloc[y].Anzahl_Mitarbeiter)
        company1.append(markus_companies.iloc[x].Name)
        company2.append(temp_table.iloc[y].Name)
    if len(lon1)>10000:
        print(x)
        print(y)
        break
    

0
1
2
3
4
5
6
7
8
9
10
11
12
12
582


In [ ]:
dummy_network_RR=pd.DataFrame(["lat1","lat2","lat1","lon2","company1","company2", "mitarbeiter1", "mitarbeiter2"])
dummy_network_RR["lat1"]=lat1
dummy_network_RR["lat2"]=lat2
dummy_network_RR["lat1"]=lat1
dummy_network_RR["lon2"]=lon2
dummy_network_RR["company1"]=company1
dummy_network_RR["company2"]=company2
dummy_network_RR["mitarbeiter1"]=mitarbeiter1
dummy_network_RR["mitarbeiter2"]=mitarbeiter2

In [ ]:
& markus_companies.Anzahl_Mitarbeiter.astype('int32',errors="ignore")>5

In [84]:
markus_companies[(markus_companies.WZ_2008_main==wz) & (markus_companies.BVD_ID!=bvID) & markus_companies.lon.notna() ].iloc[1].Anzahl_Mitarbeiter

'17'

In [91]:
markus_companies.Anzahl_Mitarbeiter[7]

'3'

In [68]:
len(lat1)

0

In [50]:
markus_companies.iloc[0].BVD_ID

'DE5210185629'

In [53]:
bvID=markus_companies.iloc[1].BVD_ID
wz=markus_companies.iloc[1].WZ_2008_main

In [56]:
markus_companies[(markus_companies.WZ_2008_main==wz) & (markus_companies.BVD_ID!=bvID) & markus_companies.lon.notna() & markus_companies.Anzahl_Mitarbeiter>15]

,Name,BVD_ID,PLZ,Ort,Strasse,HNR,HNR_Zusatz,web_adresse,Anzahl_Mitarbeiter,Anzahl_Mitarbeiter_zuletzt_Jahr,...,NACE_main_code,NACE_minor,NACE_minor_code,lat,lon,Kreis_AGS,Kreis,NUTS,Gemeinde_AGS,Gemeinde
id,,,,,,,,,,,,,,,,,,,,,
9184,FOODPOL CONCEPT CATERING GmbH,DE5210339698,40667,Meerbusch,Böhler Str.,1,NaN,www.foodpolcatering.com,10,2022,...,Event-Caterer,5620,Caterer und Erbringung sonstiger Verpflegungsd...,51.2364788,6.7030647,5111.0,Düsseldorf,DEA11,05111000,Düsseldorf
7987,noi! Event & Catering GmbH & Co. KG,DE5230350201,41238,Mönchengladbach,Monforts Quartier,1,NaN,www.noi-events.de,25,2021,...,Event-Caterer,5610,"Restaurants, Gaststätten, Imbissstuben, Cafés,...",51.1829489,6.4492271,5116.0,Mönchengladbach,DEA15,05116000,Mönchengladbach
15784,Metzgerei Peltzer e.K. Inhaber Heinz Theo Rahmen,DE5230209086,41238,Mönchengladbach,Tackhütte,82,NaN,NaN,1,2019,...,Event-Caterer,4722,Einzelhandel mit Fleisch und Fleischwaren,51.1737945,6.4836664,5116.0,Mönchengladbach,DEA15,05116000,Mönchengladbach
51734,noi! Event & Catering GmbH & Co. KG,DE5230350201-1004,41066,Mönchengladbach,Flughafenstr.,100,NaN,NaN,NaN,NaN,...,Event-Caterer,5610,"Restaurants, Gaststätten, Imbissstuben, Cafés,...",51.2299626,6.4888171,5116.0,Mönchengladbach,DEA15,05116000,Mönchengladbach
57695,Salinas Gastronomie & Event GmbH,DE5230428455,41061,Mönchengladbach,Johann-Peter-Boelling-Platz,3,NaN,NaN,2,2021,...,Event-Caterer,5600,Gastronomie,51.1928572,6.4348424,5116.0,Mönchengladbach,DEA15,05116000,Mönchengladbach
12405,Tailored-Food and Fitness GmbH,DE5270264567,41540,Dormagen,Knechtstedener Str.,14,a,www.tailored-foodandfitness.com,5,2021,...,Event-Caterer,4729,Sonstiger Einzelhandel mit Nahrungs- und Genus...,51.0987611,6.81452924048528,5162.0,Rhein-Kreis Neuss,DEA1D,05162004,Dormagen
20601,Altes Zollhaus GmbH,DE5050823249-1000,41541,Dormagen,Rheinstr.,16,NaN,NaN,NaN,NaN,...,Event-Caterer,5600,Gastronomie,51.12160645,6.85140125609356,5162.0,Rhein-Kreis Neuss,DEA1D,05162004,Dormagen
52919,Parea Service Limited,DE3370119172,41540,Dormagen,Fichtenweg,4,NaN,NaN,1,2021,...,Event-Caterer,5600,Gastronomie,51.1025201,6.80831364297986,5162.0,Rhein-Kreis Neuss,DEA1D,05162004,Dormagen
5803,REBEL Marketing GmbH,DE2012226492,41564,Kaarst,Westfalenstr.,34,NaN,www.rebel-media.de,2,2020,...,Event-Caterer,6201,Programmierungstätigkeiten,51.2129771,6.5767669,5162.0,Rhein-Kreis Neuss,DEA1D,05162016,Kaarst


In [1]:
import pandas as pd

In [34]:
df= pd.read_csv(r"C:\Users\PaulSelzner\OneDrive - rwth-aachen.de\05_Rolle\03_Entwicklung\Metabase\Daten\Markus_companies_enhanced.csv")

In [39]:
df.WZ_2008_main

0        G45110
1        I56210
2        G45100
3        G46152
4        G47761
          ...  
66972    F43220
66973    M70109
66974    C18120
66975    M70109
66976    J61909
Name: WZ_2008_main, Length: 66977, dtype: object

In [36]:
df.columns

Index(['id', 'Name', 'BVD_ID', 'PLZ', 'Ort', 'Strasse', 'HNR', 'HNR_Zusatz',
       'web_adresse', 'Anzahl_Mitarbeiter', 'Anzahl_Mitarbeiter_zuletzt_Jahr',
       'Anzahl_Mitarbeiter_zuletzt', 'Tochtergesellschaften', 'Gesellschafter',
       'Unternehmen_im_Konzern', 'Anzahl_Geschaeftsfuehrer', 'Umsatz_tsd',
       'Kapital', 'Gruendung', 'Gesellschafterkapital', 'Rechtsform',
       'Gesellschafter2', 'Gesellschafter_3', 'WZ_2008_4_main',
       'Vergleichsgruppe_Name', 'WZ_2008_4_main_description',
       'WZ_2008_3_main_description', 'WZ_2008_3_main',
       'WZ_2008_1_main_description', 'WZ_2008_1_main',
       'WZ_2008_main_description', 'WZ_2008_main', 'Vergleichsgruppe_size',
       'Allgemeines', 'Taetigkeitsbeschreibung', 'Crefonummer', 'UID_Nummer',
       'Steuernummer', 'Handelsregisternummer', 'Handelsregisterart',
       'Registernummer', 'Amtsgericht_PLZ', 'NACE_main', 'NACE_main_code',
       'NACE_minor', 'NACE_minor_code', 'lat', 'lon', 'Kreis_AGS', 'Kreis',
       '

In [43]:
sum_rela=0
for x in df.WZ_2008_main.unique():
    l=len(df[df.WZ_2008_main==x])
    if l>1:
        sum_rela+=(l*(l-1))

In [42]:
len(df[df.WZ_2008_main=="G45110"])

250

In [44]:
sum_rela

51252622

In [46]:
sum_man=0
crefos=markus_person.MAN_Crefonummer.unique()
for x in crefos:
    l=len(markus_person[markus_person.MAN_Crefonummer==x])
    if l>1:
        sum_man+=l-1
        

KeyboardInterrupt: 

In [ ]:
sum_man

In [47]:
len(markus_person.MAN_Crefonummer.unique())

69286

In [49]:
df

,id,Name,BVD_ID,PLZ,Ort,Strasse,HNR,HNR_Zusatz,web_adresse,Anzahl_Mitarbeiter,...,NACE_main_code,NACE_minor,NACE_minor_code,lat,lon,Kreis_AGS,Kreis,NUTS,Gemeinde_AGS,Gemeinde
0,1332,Lueg sportivo GmbH,DE5210185629,40667,Meerbusch,Düsseldorfer Str.,217.0,NaN,www.lueg-sportivo.de,20.0,...,Handel mit Kraftwagen mit einem Gesamtgewicht ...,4520.0,Instandhaltung und Reparatur von Kraftwagen,51.238595,6.707430,5111.0,Düsseldorf,DEA11,5111000.0,Düsseldorf
1,1424,BHG.BROICHCATERING GMBH,DE5210167614,40667,Meerbusch,Böhler Str.,1.0,NaN,www.broich-catering.com,170.0,...,Event-Caterer,5620.0,Caterer und Erbringung sonstiger Verpflegungsd...,51.236479,6.703065,5111.0,Düsseldorf,DEA11,5111000.0,Düsseldorf
2,3169,"""Autohaus Krüger & Schellenberg GmbH""",DE5210057589,40667,Meerbusch,Böhler Str.,4.0,NaN,www.suzuki-kus.de,27.0,...,Handel mit Kraftwagen,3317.0,Reparatur und Instandhaltung von Fahrzeugen a n g,51.236479,6.703065,5111.0,Düsseldorf,DEA11,5111000.0,Düsseldorf
3,3329,MBP Service GmbH,DE5210362930,40667,Meerbusch,Böhler Str.,1.0,NaN,www.mbp-service.de,9.0,...,"Handelsvermittlung von Möbeln, Einrichtungs- u...",NaN,NaN,51.236479,6.703065,5111.0,Düsseldorf,DEA11,5111000.0,Düsseldorf
4,4060,Bogie GmbH,DE5210049211,40667,Meerbusch,Düsseldorfer Str.,197.0,NaN,www.bogies.de,45.0,...,"Einzelhandel mit Blumen, Pflanzen, Sämereien, ...",8130.0,Garten- und Landschaftsbau sowie Erbringung vo...,51.238953,6.705792,5111.0,Düsseldorf,DEA11,5111000.0,Düsseldorf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66972,10952,AFDE GmbH,DE5190760993,50259,Pulheim,Im Kamp,NaN,NaN,www.afde.de,3.0,...,"Gas-, Wasser-, Heizungs- sowie Lüftungs- und K...",4120.0,Bau von Gebäuden,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66973,34254,"Franz Davids, Loherhof Verwaltungs-GmbH",DE5010310738,52511,Geilenkirchen,Gut Hommerschen,NaN,NaN,www.franz-davids.de,5.0,...,Verwaltung und Führung von Unternehmen und Bet...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66974,12066,Klinkhammer Druck e.K.,DE5030483129,53881,Euskirchen,Grondahls Mühle,12.0,e,www.druckerei-klinkhammer.de,4.0,...,Drucken a n g,7311.0,Werbeagenturen,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66975,32331,Euregio Tours Verwaltungs GmbH,DE5010465750,52249,Eschweiler,Reigate-u.-Banstead-Platz,1.0,NaN,www.euregiotours.de,1.0,...,Verwaltung und Führung von Unternehmen und Bet...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Creating Company Network
-all companies connected by a person are written in one line

In [64]:
company_network=pd.DataFrame(columns=["Name_1", "Handelsregisternummer_1","lat_1","lon_1","Name_2", "Handelsregisternummer_2","lat_2","lon_2","MAN_Crefonummer","MAN_Name"])

In [141]:
company_network=pd.DataFrame(columns=["Name_1", "Handelsregisternummer_1","lat_1","lon_1","Name_2", "Handelsregisternummer_2","lat_2","lon_2","MAN_Crefonummer","MAN_Name"])
company_network["num_connections"]=1
crefos=markus_person.MAN_Crefonummer.unique()
i=0
for x in crefos:
    temp=markus_person[markus_person.MAN_Crefonummer==x]
    l=len(temp)
    if l>1:
        for row in range(l):
            for row_2 in range(row+1,l):
                company_network.loc[len(company_network)]=[df[df.id==temp.iloc[row]["id"]].iloc[0]["Name"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["Handelsregisternummer"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["lat"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["lon"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["Name"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["Handelsregisternummer"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["lat"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["lon"],
                                                           x,
                                                           temp.iloc[0]["MAN_Name"]]
    i+=1
    if i%1000==0:
        print(i)
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000


In [172]:
for cons in range(len(company_network)):
    
    l=len(company_network[(company_network.Handelsregisternummer_1==company_network.iloc[cons].Handelsregisternummer_1) &
                         (company_network.Handelsregisternummer_2==company_network.iloc[cons].Handelsregisternummer_2)])
    company_network.iloc[cons,10]=l
    

In [173]:
company_network.to_csv("company_network_by_manager.csv", encoding="utf-8", index=False)

# Create Patent Network

In [1]:
import pandas as pd
patstat=pd.read_csv(r"C:\Users\PaulSelzner\Downloads\data-1694761486169.csv")

In [82]:
network_docdb=patstat.groupby(["doc_db_family_id"]).count()["appln_id"].index[patstat.groupby(["doc_db_family_id"]).count()["appln_id"].values>1]
network_subset_patstat=patstat[patstat.doc_db_family_id.isin(network_docdb.values)].copy()

In [83]:
network_subset_patstat

,doc_db_family_id,appln_id,appln_filing_year,cpc_class_symbol,person_id,person_name,person_address,person_ctry_code,nuts,nuts_level,han_name,han_id
31,43757822,328601593,2010,E04H 12/085,220,SIEMENS AKTIENGESELLSCHAFT,"Wittelsbacherplatz 2,80333 München",DE,DE212,3,SIEMENS AG,2709504
50,7872168,17435376,1999,F01L 9/20,220,SIEMENS AKTIENGESELLSCHAFT,"Wittelsbacherplatz 2,80333 München",DE,DE212,3,SIEMENS AG,2709504
53,7896166,17394172,1999,H01B 19/04,220,SIEMENS AKTIENGESELLSCHAFT,"Wittelsbacherplatz 2,80333 München",DE,DE212,3,SIEMENS AG,2709504
91,36001020,16259863,2005,B23P 6/007,220,SIEMENS AKTIENGESELLSCHAFT,"Wittelsbacherplatz 2,80333 München",DE,DE212,3,SIEMENS AG,2709504
119,27544041,15772483,2000,B63H 1/12,220,SIEMENS AKTIENGESELLSCHAFT,"Wittelsbacherplatz 2,80333 München",DE,DE212,3,SIEMENS AG,2709504
...,...,...,...,...,...,...,...,...,...,...,...,...
851099,83507716,580880906,2021,B23K 26/0643,79102603,Evekinger Rohr- und Profilwerke GmbH,NaN,DE,DE,0,EVEKINGER ROHR & PROFILWERKE GMBH,4506288
851106,71994331,535863965,2020,G01N 21/41,79333192,"Tomocube, Inc.","2nd Fl.,48 Yuseong-daero 1184beon-gil,Yuseong-...",KR,NaN,9,TOMOCUBE INC,3815491
851130,46881079,375711922,2012,G06V 10/54,79426111,"Indra Sistemas, S.A.","Av. de Bruselas, 35,28108 Alcobendas, Madrid",ES,ES300,4,INDRA SISTEMAS SA,1386484
851132,60785954,488192683,2017,B01D 53/78,79440970,Airovation Technologies Ltd,"2 Ilan Ramon St.,Ness Ziona",IL,NaN,9,AIROVATION TECH LTD,4580414


In [171]:
from geopy.geocoders import Nominatim
from time import sleep
import numpy
geocoder = Nominatim(user_agent = 'selzner@time.rwth-aachen.de')
#network_subset_patstat["lat"]=pd.Series()
#network_subset_patstat["lat"]=pd.Series()
z=len(network_subset_patstat[network_subset_patstat.lat.notna()].person_address.unique())
for x in network_subset_patstat[network_subset_patstat.lat.isna()].person_address.unique():
#for x in network_subset_patstat[network_subset_patstat.lat=="error"].person_address.unique():    
    if x=="" or x == " ":
        continue
        
    temp=network_subset_patstat[network_subset_patstat.person_address==x]
    if len(temp)==0:
        continue
    if temp.iloc[0].person_ctry_code!= "DE":
        continue
    else:
        print(x)
        
        if "," in x:
            adress_strings=x.split(',')
            for k in adress_strings:
                if "Postfach" in k:
                    adress_strings.remove(k)
            if len(adress_strings)==1:
                adress_strings=["",adress_strings[0]]
            if len(adress_strings)>2:
                a=adress_strings[0]
                b=adress_strings[-1]
                adress_strings=[a,b]
            
            t=adress_strings[1].split(" ")
            plz=t[0]
            if plz[0:2]=="D-":
                plz=plz[2:]
            city=""
            for u in t[1:]:
                city+=u +" "
            street_nr=adress_strings[0].split(" ")
            if len(street_nr)==2:
                street, hnr= street_nr
            if len(street_nr)==1:
                street=street_nr[0]
            if len(street_nr)>2:  
                for i, f in enumerate(street_nr):
                    if f.isnumeric():
                        break
                hnr=""
                for y in street_nr[i:]:
                    hnr+=str(y)+' '
                street=""
                for y in street_nr[:i]:
                    street+=str(y)+' '
        else:
            t =x.split(" ")
            plz=t[0]
            city=""
            for u in t[1:]:
                city+=u +" "
            hnr=""
            street=""
            if plz[0:2]=="D-":
                plz=plz[2:]
        
    current_adress={"street":f"{hnr} {street}", "city":city, "country" : "Germany", "postalcode" : plz}
    try:
        geolocation=geocoder.geocode(current_adress, country_codes="de")
    except:
        df.loc[i,"lat"]="timeout"
        df.loc[i,"lon"]="timeout"
        sleep(5)
        print(f"timeout error at {x}")
        
    try:
        network_subset_patstat.loc[network_subset_patstat["person_address"]==x,["lat"]]=geolocation.latitude
        network_subset_patstat.loc[network_subset_patstat["person_address"]==x,["lon"]]=geolocation.longitude
    except:
        network_subset_patstat.loc[network_subset_patstat["person_address"]==x,["lat"]]="error"
        network_subset_patstat.loc[network_subset_patstat["person_address"]==x,["lon"]]="error"
    sleep(1)
    z+=1
    if z%1000==0:
        print(f"{z} of {7000} % abgeschlossen.")
    
    
network_subset_patstat.to_csv(r"C:\Users\PaulSelzner\OneDrive - rwth-aachen.de\05_Rolle\03_Entwicklung\Metabase\Daten\patstat_network_companies_geolocated.csv")

In [173]:
network_subset_patstat[network_subset_patstat.lat.notna() & network_subset_patstat.lat != "error"]

,doc_db_family_id,appln_id,appln_filing_year,cpc_class_symbol,person_id,person_name,person_address,person_ctry_code,nuts,nuts_level,han_name,han_id,lat,lon
31,43757822,328601593,2010,E04H 12/085,220,SIEMENS AKTIENGESELLSCHAFT,"Wittelsbacherplatz 2,80333 München",DE,DE212,3,SIEMENS AG,2709504,48.143884,11.576315
50,7872168,17435376,1999,F01L 9/20,220,SIEMENS AKTIENGESELLSCHAFT,"Wittelsbacherplatz 2,80333 München",DE,DE212,3,SIEMENS AG,2709504,48.143884,11.576315
53,7896166,17394172,1999,H01B 19/04,220,SIEMENS AKTIENGESELLSCHAFT,"Wittelsbacherplatz 2,80333 München",DE,DE212,3,SIEMENS AG,2709504,48.143884,11.576315
91,36001020,16259863,2005,B23P 6/007,220,SIEMENS AKTIENGESELLSCHAFT,"Wittelsbacherplatz 2,80333 München",DE,DE212,3,SIEMENS AG,2709504,48.143884,11.576315
119,27544041,15772483,2000,B63H 1/12,220,SIEMENS AKTIENGESELLSCHAFT,"Wittelsbacherplatz 2,80333 München",DE,DE212,3,SIEMENS AG,2709504,48.143884,11.576315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851099,83507716,580880906,2021,B23K 26/0643,79102603,Evekinger Rohr- und Profilwerke GmbH,NaN,DE,DE,0,EVEKINGER ROHR & PROFILWERKE GMBH,4506288,NaN,NaN
851106,71994331,535863965,2020,G01N 21/41,79333192,"Tomocube, Inc.","2nd Fl.,48 Yuseong-daero 1184beon-gil,Yuseong-...",KR,NaN,9,TOMOCUBE INC,3815491,NaN,NaN
851130,46881079,375711922,2012,G06V 10/54,79426111,"Indra Sistemas, S.A.","Av. de Bruselas, 35,28108 Alcobendas, Madrid",ES,ES300,4,INDRA SISTEMAS SA,1386484,NaN,NaN
851132,60785954,488192683,2017,B01D 53/78,79440970,Airovation Technologies Ltd,"2 Ilan Ramon St.,Ness Ziona",IL,NaN,9,AIROVATION TECH LTD,4580414,NaN,NaN


In [22]:
#Weiterarbeit mit manuell ergänztem Datensatz
import pandas as pd
df=pd.read_excel(r"C:\Users\pauls\OneDrive - rwth-aachen.de\05_Rolle\03_Entwicklung\Metabase\Daten\patstat_netzwerk_geocode_händisch.xlsx",sheet_name="patstat_network_companies_geolo")

In [23]:
for x in range(len(df)):
    if df.iloc[x].lat=="error":
        lat, lon=df.iloc[x].koord_manual.split(",")
        df.loc[x,["lat"]]=lat
        df.loc[x,["lon"]]=lon

In [32]:
patent_network=pd.DataFrame(columns=['doc_db_family_id', 'appln_id', 'appln_filing_year', 'cpc_class_symbol',
                                     'person_id', 'person_name', 'person_address', 'person_ctry_code',
                                     'nuts', 'nuts_level', 'han_name', 'han_id',"lat_1","lon_1",'person_id2', 'person_name2', 
                                     'person_address2','person_ctry_code2','nuts2', 'nuts_level2', 'han_name2', 
                                     'han_id2',"lat_2","lon_2"])
#df=network_subset_patstat[network_subset_patstat.lat.notna() & network_subset_patstat.lat != "error"].copy()
#patent_network["num_connections"]=1
patent_id=df.doc_db_family_id.unique()
i=0
for x in patent_id:
    temp=df[df.doc_db_family_id==x]
    l=len(temp)
    if l>1:
        for row in range(l):
            for row_2 in range(row+1,l):
                if temp.iloc[row]["person_address"] == temp.iloc[row_2]["person_address"]:
                    continue                
                patent_network.loc[len(patent_network)]=[df[df.id==temp.iloc[row]["id"]].iloc[0]["doc_db_family_id"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["appln_id"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["appln_filing_year"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["cpc_class_symbol"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["person_id"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["person_name"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["person_address"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["person_ctry_code"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["nuts"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["nuts_level"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["han_name"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["han_id"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["lat"],
                                                           df[df.id==temp.iloc[row]["id"]].iloc[0]["lon"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["person_id"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["person_name"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["person_address"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["person_ctry_code"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["nuts"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["nuts_level"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["han_name"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["han_id"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["lat"],
                                                           df[df.id==temp.iloc[row_2]["id"]].iloc[0]["lon"]]
                                                            
                
    i+=1
    #if i==100:break
    if i%1000==0:
        print(i)
patent_network.to_csv(r"patent_network.csv", index=False, encoding="utf-8", sep=";")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000


# Processing INKAR Data

In [12]:
import pandas as pd
df= pd.read_csv(r"Daten/INKAR.csv", sep=";", header=0)
df.drop([0],inplace=True)

In [13]:
df

,Kennziffer,Raumeinheit,Aggregat,Erwerbsfähige Bevölkerung (15 bis unter 65 Jahre),Sozialversicherungspflichtig Beschäftigte am Arbeitsort,Arbeitslose,Beschäftigtenquote,Durchschnittsalter der Bevölkerung,Gesamtwanderungssaldo,Gesamtwanderungssaldo.1,...,Bruttoinlandsprodukt je Erwerbstätigen,Anteil Bruttowertschöpfung Tertiärer Sektor,Anteil Bruttowertschöpfung Sekundärer Sektor,Anteil Bruttowertschöpfung Primärer Sektor,Bruttowertschöpfung je Erwerbstätigen Tertiärer Sektor,Bruttowertschöpfung je Erwerbstätigen Sekundärer Sektor,Bruttowertschöpfung je Erwerbstätigen Primärer Sektor,Bruttowertschöpfung je Erwerbstätigen,Unternehmensinsolvenzen,"Investitionen im Bergbau und Verarb, Gewerbe"
1,51160000.0,Mönchengladbach. Stadt,verbandsfreie Gemeinde,"167,960","100,875","14,091",57.86,43.74,1.18,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN
2,51620004.0,Dormagen. Stadt,verbandsfreie Gemeinde,"41,518","19,420","1,901",62.14,44.78,2.75,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN
3,51620008.0,Grevenbroich. Stadt,verbandsfreie Gemeinde,"40,899","22,941","2,042",63.97,44.02,4.64,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN
4,51620012.0,Jüchen. Stadt,verbandsfreie Gemeinde,"14,987","3,174",520,61.44,44.70,1.37,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN
5,51620016.0,Kaarst. Stadt,verbandsfreie Gemeinde,"26,103","9,322","1,092",60.92,46.32,4.55,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,5334.0,Städteregion Aachen,Landkreis,"370,079","217,776","24,073",55.96,42.90,0.00,1.45,...,69.56,71.34,28.44,0.21,56.29,90.43,41.08,63.01,5.65,8.53
69,5358.0,Düren,Landkreis,"170,232","85,766","10,089",59.18,44.15,0.00,5.15,...,65.76,69.62,29.29,1.09,56.47,69.71,42.20,59.56,5.04,9.67
70,5362.0,Rhein-Erft-Kreis,Landkreis,"298,611","145,556","17,923",61.77,44.13,0.00,0.30,...,82.51,68.91,30.57,0.52,65.82,109.50,43.98,74.74,6.21,29.68
71,5366.0,Euskirchen,Landkreis,"124,268","57,767","6,124",59.50,44.77,0.00,7.69,...,64.45,70.37,28.17,1.46,54.24,72.72,51.82,58.38,5.04,10.06
